In [3]:
from bart_survival import simulation

In [74]:
import importlib
import numpy as np

# creates a random number generator
rng = np.random.default_rng(1)

In [77]:
importlib.reload(simulation)
rng = np.random.default_rng(1)
x_mat = simulation.get_x_matrix(
    N = 2,
    x_vars = 5, 
    VAR_CLASS = [2,2,10,100], 
    VAR_PROB = [.6, .8, None, None],
    rng = rng
)
x_mat

array([[1.00000000e+00, 1.00000000e+00, 3.11800000e+00, 8.27700000e+01,
        5.49593688e-01],
       [0.00000000e+00, 0.00000000e+00, 4.23300000e+00, 4.09200000e+01,
        2.75591132e-02]])

In [80]:
with open("tmp.txt", "a") as f:
    f.write("\n")
    f.write(str(x_mat))

In [82]:

assert(np.allclose(x_mat, [[1.00000000e+00, 1.00000000e+00, 3.11800000e+00, 8.27700000e+01, 5.49593688e-01],[0.00000000e+00, 0.00000000e+00, 4.23300000e+00, 4.09200000e+01, 2.75591132e-02]]))

AssertionError: 

In [133]:
np.exp(1)

2.718281828459045

In [191]:
np.log(1.5)

0.4054651081081644

In [371]:
importlib.reload(simulation)
rng = np.random.default_rng(2)
# x_mat = np.array([[1],[1],[1],[1]])
x_mat = np.repeat([1,0],10).reshape(-1,1)



event_dict, sv_true, sv_scale_true = simulation.simulate_survival(
    x_mat = x_mat,
    scale_f = "np.exp(1 + 1.5*x_mat[:,0])",
    shape_f = "1",
    eos = None,
    cens_scale=None,
    time_scale=None,
    true_only=False,
    rng = rng
)

assert (
    np.allclose(sv_true["sv_true"][0], np.exp(-np.cumsum(sv_true["hz_true"], axis = 1))[0])
)

# check HR returned == expected 
# event_dict, sv_true, sv_scale_true

mean shape 1.0
mean scale 7.450387894581262
mean time draws 8.556659855351331


In [392]:
# Check that HR returned is correct
importlib.reload(simulation)
rng = np.random.default_rng(2)
# x_mat = np.array([[1],[1],[1],[1]])
x_mat = np.repeat([1,0],10).reshape(-1,1)



event_dict, sv_true, sv_scale_true = simulation.simulate_survival(
    x_mat = x_mat,
    scale_f = "np.exp(1 + 1.5*x_mat[:,0])",
    shape_f = "1",
    eos = None,
    cens_scale=None,
    time_scale=None,
    true_only=False,
    rng = rng
)

hz = sv_true["hz_true"]
# print(hz)
hr = hz[1,:]/hz[10,:]
print(hr[0])
exp_hr = np.power(np.exp(-1.5), 1)
print(exp_hr)
assert(np.allclose(hr[0],exp_hr))
# assert (
    # np.allclose(sv_true["sv_true"][0], np.exp(-np.cumsum(sv_true["hz_true"], axis = 1))[0])
# )

mean shape 1.0
mean scale 7.450387894581262
mean time draws 8.556659855351331
0.22313016014842985
0.22313016014842982


In [426]:
# Check that eos and time_scale work correctly
importlib.reload(simulation)
rng = np.random.default_rng(2)
# x_mat = np.array([[1],[1],[1],[1]])
x_mat = np.repeat([1,0],10).reshape(-1,1)

event_dict, sv_true, sv_scale_true = simulation.simulate_survival(
    x_mat = x_mat,
    scale_f = "np.exp(4 + 1.5*x_mat[:,0])",
    shape_f = "1",
    eos = 180,
    cens_scale=None,
    time_scale=30,
    true_only=False,
    rng = rng
)
# eos set to 180
assert(event_dict["t_event"].max() == 180)
ev_msk = event_dict["t_event"]==180
assert(all(event_dict["status"][ev_msk] == False) == True)

# check scaling worked
tdiff = sv_scale_true["true_times"][1] - sv_scale_true["true_times"][0] 
assert(tdiff == 30)
assert(sv_scale_true["scale_times"].shape[0] == 180/30)

mean shape 1.0
mean scale 149.64504114868228
mean time draws 171.86510746381683
(20, 180)


In [427]:
sv_true["true_times"]

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180])

In [428]:
sv_scale_true

{'sv_true': array([[0.88461466, 0.78254309, 0.69224909, 0.61237369, 0.54171475,
         0.4792088 ],
        [0.88461466, 0.78254309, 0.69224909, 0.61237369, 0.54171475,
         0.4792088 ],
        [0.88461466, 0.78254309, 0.69224909, 0.61237369, 0.54171475,
         0.4792088 ],
        [0.88461466, 0.78254309, 0.69224909, 0.61237369, 0.54171475,
         0.4792088 ],
        [0.88461466, 0.78254309, 0.69224909, 0.61237369, 0.54171475,
         0.4792088 ],
        [0.88461466, 0.78254309, 0.69224909, 0.61237369, 0.54171475,
         0.4792088 ],
        [0.88461466, 0.78254309, 0.69224909, 0.61237369, 0.54171475,
         0.4792088 ],
        [0.88461466, 0.78254309, 0.69224909, 0.61237369, 0.54171475,
         0.4792088 ],
        [0.88461466, 0.78254309, 0.69224909, 0.61237369, 0.54171475,
         0.4792088 ],
        [0.88461466, 0.78254309, 0.69224909, 0.61237369, 0.54171475,
         0.4792088 ],
        [0.57725616, 0.33322467, 0.19235599, 0.11103868, 0.06409776,
         0

0.9320390859672263

In [370]:
# sv_scale_true["sv_true"][:,sv_scale_true["true_times"].tolist()]
# sv_scale_true["sv_true"][:,1]
# sv_scale_true["sv_true"].shape
print(sv_true["sv_true"].shape)
print(sv_true["hz_true"].shape)
print(sv_true["true_times"])

print(sv_true["sv_true"][0])
print(np.exp(-np.cumsum(sv_true["hz_true"], axis = 1))[0])

assert (
    np.allclose(sv_true["sv_true"][0], np.exp(-np.cumsum(sv_true["hz_true"], axis = 1))[0])
)
# sv_true["sv_true"]

(20, 36)
(20, 36)
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36]
[0.92119366 0.84859775 0.78172286 0.72011814 0.66336826 0.61109064
 0.56293282 0.51857014 0.47770352 0.44005745 0.40537813 0.37343176
 0.34400297 0.31689336 0.29192015 0.26891499 0.24772278 0.22820065
 0.21021699 0.19365056 0.17838967 0.16433143 0.15138107 0.13945128
 0.12846164 0.11833804 0.10901226 0.1004214  0.09250756 0.08521737
 0.0785017  0.07231527 0.06661637 0.06136658 0.0565305  0.05207554]
[0.92119366 0.84859775 0.78172286 0.72011814 0.66336826 0.61109064
 0.56293282 0.51857014 0.47770352 0.44005745 0.40537813 0.37343176
 0.34400297 0.31689336 0.29192015 0.26891499 0.24772278 0.22820065
 0.21021699 0.19365056 0.17838967 0.16433143 0.15138107 0.13945128
 0.12846164 0.11833804 0.10901226 0.1004214  0.09250756 0.08521737
 0.0785017  0.07231527 0.06661637 0.06136658 0.0565305  0.05207554]


In [347]:
x_mat = np.array([0,1]).reshape(-1,1)
scale_f = "np.exp(2+.5*x_mat[:,0])"
print(eval(scale_f))
scale = eval(scale_f)
shape = np.array([2,2])
t = np.arange(1,10,1)
hz = simulation.weibull_hazard(scale, shape, t)
hz_sv = np.exp(-np.cumsum(hz, axis = 1))
sv = simulation.weibull_survival(scale, shape, t)
# print(sv)
# print(hz_sv)
# print(hz)

hr = hz[1,:]/hz[0,:]
print(hr)
np.power(np.exp(-.5), 2)



[ 7.3890561  12.18249396]
[0.36787944 0.36787944 0.36787944 0.36787944 0.36787944 0.36787944
 0.36787944 0.36787944 0.36787944]


0.36787944117144233

In [218]:
sv_true["sv_true"][0]

array([0.95143199, 0.95143199, 0.90522284, 0.86125797, 0.81942838,
       0.77963038, 0.74176529, 0.70573923, 0.67146288, 0.63885126,
       0.60782353, 0.57830275, 0.55021574, 0.52349286, 0.49806785,
       0.47387769, 0.4508624 , 0.42896491, 0.40813094, 0.38830883,
       0.36944945, 0.35150602, 0.33443407, 0.31819128, 0.30273736,
       0.28803401, 0.27404477, 0.26073497, 0.24807159, 0.23602325,
       0.22456007, 0.21365363, 0.2032769 , 0.19340415, 0.18401089,
       0.17507385, 0.16657086, 0.15848085, 0.15078375, 0.14346048,
       0.13649289, 0.1298637 , 0.12355648, 0.11755559, 0.11184615,
       0.10641401, 0.10124569, 0.09632839, 0.09164991, 0.08719866,
       0.08296359, 0.07893422, 0.07510054, 0.07145305, 0.06798272,
       0.06468094, 0.06153951, 0.05855066, 0.05570697, 0.0530014 ,
       0.05042722, 0.04797807, 0.04564787, 0.04343085, 0.0413215 ,
       0.0393146 , 0.03740516, 0.03558847, 0.03386001, 0.0322155 ,
       0.03065085, 0.0291622 , 0.02774585, 0.02639829, 0.02511

In [197]:
importlib.reload(simulation)
simulation.weibull_hazard(scale = np.array([20.1,7.38]), shape=np.array([1,1]), t=np.array([1,2]))

array([[0.04975124, 0.04975124],
       [0.13550136, 0.13550136]])

In [201]:
# np.exp(.28/.5)
# np.log(.135/0.03)
# np.log(.04/.135)
np.log(20.1/ 7.5)


0.9858167945227654

In [154]:
np.array([10,20]).reshape(-1,1)*np.power(np.array([1,2])/np.array([10,20]).reshape(-1,1), np.array([1,1]).reshape(-1,1)-1)

array([[10., 10.],
       [20., 20.]])

In [84]:
event_dict, sv_true, sv_scale_true

({'t_event': array([[14.],
         [ 2.],
         [30.],
         [ 2.]]),
  't_event_scale': array([[2.],
         [1.],
         [3.],
         [1.]]),
  'status': array([[1.],
         [1.],
         [0.],
         [1.]]),
  'cens_percentage': None},
 {'sv_true': array([[0.95143199, 0.90667782, 0.86402884, 0.823386  , 0.78465496,
          0.74774577, 0.71257275, 0.67905422, 0.64711236, 0.616673  ,
          0.58766548, 0.56002243, 0.53367968, 0.50857605, 0.48465327,
          0.46185579, 0.44013067, 0.41942748, 0.39969814, 0.38089684,
          0.36297993, 0.34590581, 0.32963483, 0.31412922, 0.29935298,
          0.28527179, 0.27185296, 0.25906534, 0.24687923, 0.23526634,
          0.2241997 , 0.21365363],
         [0.95143199, 0.90667782, 0.86402884, 0.823386  , 0.78465496,
          0.74774577, 0.71257275, 0.67905422, 0.64711236, 0.616673  ,
          0.58766548, 0.56002243, 0.53367968, 0.50857605, 0.48465327,
          0.46185579, 0.44013067, 0.41942748, 0.39969814, 0.38089684

In [64]:
# test check_inputs
# u, scale, shape, t exceptions
# These all throw errors correctly
try:
    simulation.check_inputs(scale = -1, shape = -1, u =-0.1, t=-1)
except Exception as e:
    print(e)
    errors = str(e)
    
# simulation.check_inputs(scale = 1, shape = -1, u = 0.1, t=1)
# simulation.check_inputs(scale = -1, shape = 1, u = 0, t=1)
# simulation.check_inputs(scale = -1, shape = 1, u = 0.1, t=-1)

['u must be between 0 and 1', 'scale must be > 0', 'shape must be > 0', 't must be > 0']


In [58]:
with open("tmp.txt", "a") as f:
    f.write(str(errors))

In [71]:
try:
        simulation.check_inputs(scale = -1, shape = -1, u = -0.1, t=-1)
except Exception as e:
        error = str(e)
print(error)
assert (error == "['u must be between 0 and 1', 'scale must be > 0', 'shape must be > 0', 't must be > 0']")


['u must be between 0 and 1', 'scale must be > 0', 'shape must be > 0', 't must be > 0']


In [38]:
# Check Survival components
importlib.reload(simulation)

iws = simulation.inverse_weibull_survival(scale = 1,shape =2, u = np.array([.1,.5,.9]))
wh = simulation.weibull_hazard(scale=1, shape=2, t=np.array([1,2]))
ws = simulation.weibull_survival(scale=1, shape=2, t=np.array([1,2]))
# event_dict, sv_true, sv_scale_true

iws, wh, ws

(array([[1.51742713],
        [0.83255461],
        [0.32459285]]),
 array([[2., 4.]]),
 array([[0.36787944, 0.01831564]]))

In [45]:
assert (np.allclose(iws, np.array([[1.51742713],[0.83255461],[0.32459285]])))
assert (np.allclose(wh, np.array([[2., 4.]])))
assert (np.allclose(ws, np.array([[0.36787944, 0.01831564]])))